In [1]:
import os
import copy
import torch
import detectron2
from detectron2.data import detection_utils as utils
from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator
from detectron2.data import build_detection_test_loader, build_detection_train_loader

import numpy as np
import random

In [2]:
TRAINJSON = "./dataset/train.json"
TESTJSON = "./dataset/test.json"
IMAGEROOT = "./dataset/"

TRAIN_DATASET_NAME = 'coco_trash_train'
TEST_DATASET_NAME = 'coco_trash_test'

TARGETCLASSES = ["General trash", "Paper", "Paper pack", "Metal", "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]

MODELZOO_CONFIGNAME = 'COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml'

NUM_WORKERS = 4
IMS_PER_BATCH = 4
BASE_LR = 0.001
MAX_ITER = 15000
STEPS = (8000,12000)
GAMMA = 0.005
CHECKPOINT_PERIOD = 3000

SEED = 42

OUTPUT_DIR = "./output"

ROI_BATCH = 128
ROI_NUM_CLASSES = len(TARGETCLASSES)

TEST_EVAL_PERIOD = 3000

print(len(TARGETCLASSES))

10


In [3]:
def setSeed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    print(f">> Seed : {seed}")
setSeed(SEED)

>> Seed : 42


In [4]:
# Register Dataset
try:
    register_coco_instances(TRAIN_DATASET_NAME, {}, TRAINJSON, IMAGEROOT)
except AssertionError:
    pass

try:
    register_coco_instances(TEST_DATASET_NAME, {}, TESTJSON, IMAGEROOT)
except AssertionError:
    pass

MetadataCatalog.get(TRAIN_DATASET_NAME).thing_classes = TARGETCLASSES

In [5]:
# config 불러오기
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(MODELZOO_CONFIGNAME))

In [6]:
# config 수정하기
cfg.DATASETS.TRAIN = (TRAIN_DATASET_NAME,)
cfg.DATASETS.TEST = (TEST_DATASET_NAME,)

cfg.DATALOADER.NUM_WOREKRS = NUM_WORKERS

cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(MODELZOO_CONFIGNAME)

cfg.SOLVER.IMS_PER_BATCH = IMS_PER_BATCH
cfg.SOLVER.BASE_LR = BASE_LR
cfg.SOLVER.MAX_ITER = MAX_ITER
cfg.SOLVER.STEPS = STEPS
cfg.SOLVER.GAMMA = GAMMA
cfg.SOLVER.CHECKPOINT_PERIOD = CHECKPOINT_PERIOD

cfg.OUTPUT_DIR = OUTPUT_DIR

# cfg.SEED = SEED

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = ROI_BATCH
cfg.MODEL.ROI_HEADS.NUM_CLASSES =ROI_NUM_CLASSES

cfg.TEST.EVAL_PERIOD = TEST_EVAL_PERIOD

In [7]:
# mapper - input data를 어떤 형식으로 return할지 (따라서 augmnentation 등 데이터 전처리 포함 됨)
import detectron2.data.transforms as T

def MyMapper(dataset_dict):
    dataset_dict = copy.deepcopy(dataset_dict)
    image = utils.read_image(dataset_dict['file_name'], format='BGR')
    
    transform_list = [
        T.RandomFlip(prob=0.5, horizontal=False, vertical=True),
        T.RandomBrightness(0.8, 1.8),
        T.RandomContrast(0.6, 1.3)
    ]
    
    image, transforms = T.apply_transform_gens(transform_list, image)
    
    dataset_dict['image'] = torch.as_tensor(image.transpose(2,0,1).astype('float32'))
    
    annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop('annotations')
        if obj.get('iscrowd', 0) == 0
    ]
    
    instances = utils.annotations_to_instances(annos, image.shape[:2])
    dataset_dict['instances'] = utils.filter_empty_instances(instances)
    
    return dataset_dict

In [8]:
# trainer - DefaultTrainer를 상속
class MyTrainer(DefaultTrainer):
    
    @classmethod
    def build_train_loader(cls, cfg, sampler=None):
        return build_detection_train_loader(
        cfg, mapper = MyMapper, sampler = sampler
        )
    
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            os.makedirs('./output_eval', exist_ok = True)
            output_folder = './output_eval'
            
        return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [9]:
# train
os.makedirs(cfg.OUTPUT_DIR, exist_ok = True)

trainer = MyTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[09/29 04:52:05 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (11, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (11,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (40, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (40,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias

[09/29 04:52:06 d2.engine.train_loop]: Starting training from iteration 0


/opt/ml/detection/detectron2/detectron2/modeling/roi_heads/fast_rcnn.py:103: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370172916/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  num_fg = fg_inds.nonzero().numel()


[09/29 04:52:23 d2.utils.events]:  eta: 2:12:03  iter: 19  total_loss: 3.325  loss_cls: 2.439  loss_box_reg: 0.6966  loss_rpn_cls: 0.1062  loss_rpn_loc: 0.04659  time: 0.5304  data_time: 0.0323  lr: 1.9981e-05  max_mem: 20741M
[09/29 04:52:33 d2.utils.events]:  eta: 2:11:43  iter: 39  total_loss: 2.783  loss_cls: 1.986  loss_box_reg: 0.688  loss_rpn_cls: 0.0424  loss_rpn_loc: 0.02551  time: 0.5290  data_time: 0.0139  lr: 3.9961e-05  max_mem: 20741M
[09/29 04:52:44 d2.utils.events]:  eta: 2:11:30  iter: 59  total_loss: 2.052  loss_cls: 1.198  loss_box_reg: 0.747  loss_rpn_cls: 0.04386  loss_rpn_loc: 0.02164  time: 0.5287  data_time: 0.0139  lr: 5.9941e-05  max_mem: 20741M
[09/29 04:52:55 d2.utils.events]:  eta: 2:11:20  iter: 79  total_loss: 1.611  loss_cls: 0.8115  loss_box_reg: 0.6789  loss_rpn_cls: 0.07383  loss_rpn_loc: 0.03665  time: 0.5286  data_time: 0.0140  lr: 7.9921e-05  max_mem: 20741M
[09/29 04:53:05 d2.utils.events]:  eta: 2:11:16  iter: 99  total_loss: 1.726  loss_cls: 0.8